# Applied AI Midterm Notebook

## 1. Setup & Imports
TODO: Install required packages if needed, import torch, torchvision, etc.

In [ ]:
# TODO: Add imports
from keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input, Convolution2D, MaxPool2D, Dense, Flatten, Dropout
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
from keras.utils import plot_model

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from pathlib import Path
import pandas as pd
import numpy as np

from datetime import datetime
import os
import tensorflow as tf

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


## 2. Load Dataset and Preprocessing
- TODO: Load dataset
- TODO: Apply 128x128 resize transformation for classifiers
- TODO: Generate 32x32 LR images for SRGAN
- TODO: Show sample images

In [ ]:
# TODO: Load and preprocess datasets
img_width, img_height, img_depth = 128, 128, 3
import cv2

def load_images_from_folder(folder, img_size=(img_width,img_height)):
    images, labels = [], []
    for filename in os.listdir(folder):
        path = os.path.join(folder, filename)
        img = cv2.imread(path)
        if img is not None:
            images.append(path)
            if str(filename).startswith("cat"):
                labels.append("cat")
            elif str(filename).startswith("dog"):
                labels.append("dog")
    df = pd.DataFrame({
        'Image': images,
        'Class': labels
    })

    return df

df = load_images_from_folder("train")

def split_dataframe_three_ways(df, train_ratio=0.6, valid_ratio=0.2, test_ratio=0.2, random_state=1):
    """
    Split DataFrame into train, validation, and test sets
    """
    assert train_ratio + valid_ratio + test_ratio == 1.0, "Ratios must sum to 1.0"
    
    # First, split out test set
    df_temp, df_test = train_test_split(
        df, 
        test_size=test_ratio, 
        random_state=random_state, 
        stratify=df['Class']
    )
    
    # Then split remaining into train and validation
    remaining_ratio = 1.0 - test_ratio
    valid_size = valid_ratio / remaining_ratio
    
    df_train, df_valid = train_test_split(
        df_temp, 
        test_size=valid_size, 
        random_state=random_state, 
        stratify=df_temp['Class']
    )
    
    return df_train, df_valid, df_test

# Usage
df_train, df_valid, df_test = split_dataframe_three_ways(df, train_ratio=0.6, valid_ratio=0.2, test_ratio=0.2)
nb_train_samples = len(df_train)
nb_valid_samples = len(df_valid)


In [ ]:
#Training parameters
epochs = 50
freq = 20
batch_size = 32
num_classes = 2
train_valid_split = 0.2

act_type = 'sigmoid'
class_mode = 'binary'
loss_fun = 'binary_crossentropy'

fold_num = 1
path = 'split_train/train'

ds_train = tf.keras.preprocessing.image_dataset_from_directory(
    path,
    labels='inferred',
    label_mode='binary',
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(img_width, img_height),
    shuffle=True,
    seed=123,
    validation_split=train_valid_split,
    subset='training'
)

ds_valid = tf.keras.preprocessing.image_dataset_from_directory(
    path,
    labels='inferred',
    label_mode='binary',
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(img_width, img_height),
    shuffle=True,
    seed=123,
    validation_split=train_valid_split,
    subset='validation'
)

# Use tf.keras.layers.preprocessing for augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(0.5),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomTranslation(0.2, 0.2)
    # Rotates by up to 90 degrees (0.5 * 180)
])

# Prepare the datasets for training and validation
# Apply data augmentation only to the training dataset
ds_train = ds_train.map(lambda x, y: (data_augmentation(x, training=True), y))

# You might want to add a rescaling layer as the first layer in your model
# instead of in the data augmentation pipeline, as it's applied to all data.
# However, for demonstration, we can include it here.
rescale = tf.keras.layers.Rescaling(1./255)

ds_train = ds_train.map(lambda x, y: (rescale(x), y))
ds_valid = ds_valid.map(lambda x, y: (rescale(x), y))

# Optimize performance
AUTOTUNE = tf.data.AUTOTUNE

ds_train = ds_train.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
ds_valid = ds_valid.cache().prefetch(buffer_size=AUTOTUNE)


## 3. Train Classifier A
- TODO: Implement transfer learning model (EfficientNet/ResNet)
- TODO: Train baseline classifier
- TODO: Save best model
- TODO: Evaluate performance (Confusion matrix, ROC, F1)

In [ ]:
#Perform transfer learning by using various pre-trained models
# Build Model
image_input = Input(shape = (img_width, img_height, img_depth))
vgg_model = applications.vgg16.VGG16(input_tensor = image_input,
                                 include_top = False,
                                 weights = 'imagenet')

vgg_output = vgg_model.layers[-1].output

flat1 = Flatten()(vgg_output)
fc1 = Dense(512, activation = 'relu')(flat1)
dropfc1 = Dropout(0.5)(fc1)
fc2 = Dense(256, activation = 'relu')(dropfc1)
dropfc2 = Dropout(0.5)(fc2)

output = Dense(1, activation = act_type)(dropfc2)

for layer in vgg_model.layers[:-5]:
    layer.trainable = False

model = Model(image_input, output)

# Compile the model
opt = Adam(learning_rate=0.00001)
model.compile(loss = loss_fun, optimizer = opt, metrics = ['accuracy'])

# Folder setup
fold_num = 1

init_time = datetime.now()
current_time = init_time.strftime('%Y%m%d_%H%M%S')
name_dir = 'models/'+'ClassA' + current_time + '_fold_num' + str(fold_num)
os.mkdir(name_dir)

tensorboard = TensorBoard(
    log_dir='./logs',          # Directory for logs
    histogram_freq=1,          # How often to compute histograms
    write_graph=True,          # Write computation graph
    write_images=True          # Write model weights as images
)


# Callbacks3: EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='auto')


figure_name = name_dir + '/model_output.png'
plot_model(model, figure_name, show_shapes = True)

# Display model
model.summary()
history = model.fit(ds_train,
                              epochs = epochs,
                              steps_per_epoch = 16000 // batch_size,
                              validation_data = ds_valid,
                              validation_steps = 4000 // batch_size,
                              verbose=1,
                              callbacks = [early_stop, tensorboard])

early_stop_name = name_dir + '/fold_num_' + str(fold_num) + 'early_stop_model.weights.h5'
model.save_weights(early_stop_name)

## 4. Build SRGAN (Generator + Discriminator)
- TODO: Implement Generator model
- TODO: Implement Discriminator model
- TODO: Perceptual loss (VGG19)
- TODO: GAN training loop with checkpoints

In [ ]:
# TODO: SRGAN model architecture


## 5. Train SRGAN
- TODO: Train for ≥150 epochs
- TODO: Save checkpoints every n epochs
- TODO: Visualize generated results across epochs

In [ ]:
# TODO: SRGAN training loop


## 6. Generate New High-Resolution Images
- TODO: Use trained SRGAN to generate HR images
- TODO: Visualize some HR generated samples
- TODO: Save generated data

In [ ]:
# TODO: Generate HR data


## 7. Train Classifier B with GAN Data
- TODO: Load generated HR data
- TODO: Train new classifier using combined or synthetic data
- TODO: Save best model
- TODO: Evaluate performance

In [ ]:
# TODO: Classifier B


## 8. Final Evaluation & Comparison
- TODO: Compare Classifier A vs Classifier B
- TODO: Display metrics table
- TODO: Plot ROC curves

In [ ]:
# TODO: Final evaluation


## 9. Save Outputs
- TODO: Export metrics, samples, and models as required